In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import LearningRateScheduler

import matplotlib.pyplot as plt


C:\Users\pmxjo1\.conda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\pmxjo1\.conda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\pmxjo1\.conda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\pmxjo1\.conda\envs\mle_tf\lib\site-packages\tensorflow\python\frame

In [2]:
df= pd.read_csv("student-mat.csv", sep=";")

In [3]:
label1=LabelEncoder()
df["school"]=label1.fit_transform(df["school"])
df["sex"]=label1.fit_transform(df["sex"])
df["address"]=label1.fit_transform(df["address"])
df["famsize"]=label1.fit_transform(df["famsize"])
df["Pstatus"]=label1.fit_transform(df["Pstatus"])
df["Medu"]=label1.fit_transform(df["Medu"])
df["Mjob"]=label1.fit_transform(df["Mjob"])
df["Fjob"]=label1.fit_transform(df["Fjob"])
df["reason"]=label1.fit_transform(df["reason"])
df["guardian"]=label1.fit_transform(df["guardian"])
df["schoolsup"]=label1.fit_transform(df["schoolsup"])
df["famsup"]=label1.fit_transform(df["famsup"])
df["paid"]=label1.fit_transform(df["paid"])
df["activities"]=label1.fit_transform(df["activities"])
df["nursery"]=label1.fit_transform(df["nursery"])
df["higher"]=label1.fit_transform(df["higher"])
df["internet"]=label1.fit_transform(df["internet"])
df["romantic"]=label1.fit_transform(df["romantic"])
df['G1'] = df['G1'].map(lambda x: str(x).replace('""','')).astype('float')
df['G2'] = df['G2'].map(lambda x: str(x).replace('""','')).astype('float')
df['G3'] = df['G3'].map(lambda x: str(x).replace('"','')).astype('float')

In [4]:
#One hot encoding

df=pd.get_dummies(df, drop_first=True, columns=["Mjob"])
df=pd.get_dummies(df, drop_first=True, columns=["Fjob"])

In [5]:
Y = df.loc[:, ["G3"]]
X = df.drop(labels=['school', 'sex','address', 'famsize', 'Pstatus', 'reason', 'guardian', 'activities','G1','G2','G3'],axis=1)

In [6]:
#Splitting data in Training 60% / Validation 20% / Test 20&

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

In [7]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
X_val=scaler.fit_transform(X_val)



In [8]:
kfold = KFold(n_splits=10) #10-fold cross-validation data split

In [9]:
acc_per_fold = [] #store the accuracy of every fold
overall_cm = np.zeros((2,2)) #Holds the overall confusion matrix

fold_no = 0 #keep track which fold the loop is in 

for train, test in kfold.split(X, Y):
    
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units=28, kernel_initializer='uniform', activation='relu', input_dim=28))
    # Adding dropout to prevent overfitting
    classifier.add(Dropout(rate=0.1))

    # Adding the second hidden layer
    classifier.add(Dense(units=28, kernel_initializer='uniform', activation='relu'))
    # Adding dropout to prevent overfitting
    classifier.add(Dropout(rate=0.1))

    # Adding the output layer
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    # Compiling the ANN
    classifier.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    # Fitting the ANN
    classifier.fit(X[train], Y[train], batch_size=100, epochs=500, verbose=0)
    
    # Predictions from our model
    prediction = classifier.predict(X[test])
    prediction = prediction > 0.5
    
    #Measuring accuracy
    cm = confusion_matrix(Y[test], prediction)
    overall_cm += cm
    accuracy = (np.trace(cm)/Y[test].size) * 100
    acc_per_fold.append(accuracy)
    print(f'Accuracy for fold {fold_no} is {accuracy}%')
    
    fold_no += 1

print(f'Average accuracy for {kfold.get_n_splits()} folds is {np.mean(acc_per_fold)}%')

Instructions for updating:
Colocations handled automatically by placer.


KeyError: "None of [Int64Index([ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49,\n            ...\n            385, 386, 387, 388, 389, 390, 391, 392, 393, 394],\n           dtype='int64', length=355)] are in the [columns]"

This was before

In [14]:
model = Sequential()
model.add(Dense(28, activation = 'relu', input_dim = 28))
model.add(Dense(units = 28, activation = 'softmax')) #With relu it increases acccurracy really fast. 
#model.add(Dense(units = 1, activation = 'linear'))
model.add(Dense(units = 1))


In [15]:
initial_learning_rate = 0.002
number_of_epochs = 1000 #Mas lento con 5000
decay = initial_learning_rate / number_of_epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)


model.compile(optimizer = 'adam',loss = tf.losses.mean_squared_error,metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = 64, epochs = number_of_epochs, callbacks=[LearningRateScheduler(lr_time_based_decay, verbose=1)])

Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.0010000000474974513.
237/237 [==============================] - 0s 1ms/step - loss: 128.5519 - accuracy: 0.0844
Epoch 2/1000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009999980475013564.
237/237 [==============================] - 0s 21us/step - loss: 128.1172 - accuracy: 0.0844
Epoch 3/1000

Epoch 00003: LearningRateScheduler setting learning rate to 0.0009999940684607064.
237/237 [==============================] - 0s 21us/step - loss: 127.6815 - accuracy: 0.0844
Epoch 4/1000

Epoch 00004: LearningRateScheduler setting learning rate to 0.000999988110387376.
237/237 [==============================] - 0s 21us/step - loss: 127.2563 - accuracy: 0.0844
Epoch 5/1000

Epoch 00005: LearningRateScheduler setting learning rate to 0.0009999800568788481.
237/237 [==============================] - 0s 21us/step - loss: 126.8417 - accuracy: 0.0844
Epoch 6/1000

Epoch 00006: LearningRateScheduler setting 

Epoch 44/1000

Epoch 00044: LearningRateScheduler setting learning rate to 0.0009981098889740038.
237/237 [==============================] - 0s 21us/step - loss: 113.4419 - accuracy: 0.0000e+00
Epoch 45/1000

Epoch 00045: LearningRateScheduler setting learning rate to 0.000998022102387259.
237/237 [==============================] - 0s 21us/step - loss: 113.1925 - accuracy: 0.0000e+00
Epoch 46/1000

Epoch 00046: LearningRateScheduler setting learning rate to 0.0009979323372692574.
237/237 [==============================] - 0s 21us/step - loss: 112.9493 - accuracy: 0.0000e+00
Epoch 47/1000

Epoch 00047: LearningRateScheduler setting learning rate to 0.000997840593631869.
237/237 [==============================] - 0s 21us/step - loss: 112.7101 - accuracy: 0.0000e+00
Epoch 48/1000

Epoch 00048: LearningRateScheduler setting learning rate to 0.0009977467550825838.
237/237 [==============================] - 0s 21us/step - loss: 112.4651 - accuracy: 0.0000e+00
Epoch 49/1000

Epoch 00049: Lear

237/237 [==============================] - 0s 21us/step - loss: 102.9997 - accuracy: 0.0000e+00
Epoch 87/1000

Epoch 00087: LearningRateScheduler setting learning rate to 0.000992546154194793.
237/237 [==============================] - 0s 21us/step - loss: 102.7727 - accuracy: 0.0000e+00
Epoch 88/1000

Epoch 00088: LearningRateScheduler setting learning rate to 0.0009923735342765522.
237/237 [==============================] - 0s 21us/step - loss: 102.5489 - accuracy: 0.0000e+00
Epoch 89/1000

Epoch 00089: LearningRateScheduler setting learning rate to 0.0009921989363364519.
237/237 [==============================] - 0s 21us/step - loss: 102.3343 - accuracy: 0.0000e+00
Epoch 90/1000

Epoch 00090: LearningRateScheduler setting learning rate to 0.0009920223603863579.
237/237 [==============================] - 0s 21us/step - loss: 102.1210 - accuracy: 0.0000e+00
Epoch 91/1000

Epoch 00091: LearningRateScheduler setting learning rate to 0.0009918438064381362.
237/237 [======================

237/237 [==============================] - 0s 21us/step - loss: 95.3998 - accuracy: 0.0000e+00
Epoch 129/1000

Epoch 00129: LearningRateScheduler setting learning rate to 0.0009836248338822688.
237/237 [==============================] - 0s 21us/step - loss: 95.2422 - accuracy: 0.0000e+00
Epoch 130/1000

Epoch 00130: LearningRateScheduler setting learning rate to 0.000983371125748188.
237/237 [==============================] - 0s 21us/step - loss: 95.0882 - accuracy: 0.0000e+00
Epoch 131/1000

Epoch 00131: LearningRateScheduler setting learning rate to 0.0009831155564676887.
237/237 [==============================] - 0s 21us/step - loss: 94.9322 - accuracy: 0.0000e+00
Epoch 132/1000

Epoch 00132: LearningRateScheduler setting learning rate to 0.0009828580096671053.
237/237 [==============================] - 0s 21us/step - loss: 94.7767 - accuracy: 0.0000e+00
Epoch 133/1000

Epoch 00133: LearningRateScheduler setting learning rate to 0.0009825986017428967.
237/237 [======================

237/237 [==============================] - 0s 25us/step - loss: 89.2967 - accuracy: 0.0000e+00
Epoch 171/1000

Epoch 00171: LearningRateScheduler setting learning rate to 0.0009713515863007244.
237/237 [==============================] - 0s 21us/step - loss: 89.1617 - accuracy: 0.0000e+00
Epoch 172/1000

Epoch 00172: LearningRateScheduler setting learning rate to 0.0009710194869075212.
237/237 [==============================] - 0s 17us/step - loss: 89.0258 - accuracy: 0.0000e+00
Epoch 173/1000

Epoch 00173: LearningRateScheduler setting learning rate to 0.0009706855850252837.
237/237 [==============================] - 0s 21us/step - loss: 88.8912 - accuracy: 0.0000e+00
Epoch 174/1000

Epoch 00174: LearningRateScheduler setting learning rate to 0.0009703498224772946.
237/237 [==============================] - 0s 21us/step - loss: 88.7560 - accuracy: 0.0000e+00
Epoch 175/1000

Epoch 00175: LearningRateScheduler setting learning rate to 0.0009700122574621261.
237/237 [=====================

...

237/237 [==============================] - 0s 21us/step - loss: 41.1026 - accuracy: 0.0380
Epoch 807/1000

Epoch 00807: LearningRateScheduler setting learning rate to 0.0005219973293434585.
237/237 [==============================] - 0s 21us/step - loss: 41.0669 - accuracy: 0.0380
Epoch 808/1000

Epoch 00808: LearningRateScheduler setting learning rate to 0.0005211561929966983.
237/237 [==============================] - 0s 21us/step - loss: 41.0306 - accuracy: 0.0380
Epoch 809/1000

Epoch 00809: LearningRateScheduler setting learning rate to 0.0005203153505778008.
237/237 [==============================] - 0s 21us/step - loss: 40.9957 - accuracy: 0.0380
Epoch 810/1000

Epoch 00810: LearningRateScheduler setting learning rate to 0.0005194748601986381.
237/237 [==============================] - 0s 21us/step - loss: 40.9607 - accuracy: 0.0380
Epoch 811/1000

Epoch 00811: LearningRateScheduler setting learning rate to 0.0005186346637435844.
237/237 [==============================] - 0s 21us

237/237 [==============================] - 0s 17us/step - loss: 39.5337 - accuracy: 0.0380
Epoch 850/1000

Epoch 00850: LearningRateScheduler setting learning rate to 0.00048614822252721914.
237/237 [==============================] - 0s 21us/step - loss: 39.5003 - accuracy: 0.0380
Epoch 851/1000

Epoch 00851: LearningRateScheduler setting learning rate to 0.00048532318085836676.
237/237 [==============================] - 0s 21us/step - loss: 39.4669 - accuracy: 0.0380
Epoch 852/1000

Epoch 00852: LearningRateScheduler setting learning rate to 0.00048449857829977225.
237/237 [==============================] - 0s 21us/step - loss: 39.4335 - accuracy: 0.0380
Epoch 853/1000

Epoch 00853: LearningRateScheduler setting learning rate to 0.0004836743857944836.
237/237 [==============================] - 0s 17us/step - loss: 39.3992 - accuracy: 0.0380
Epoch 854/1000

Epoch 00854: LearningRateScheduler setting learning rate to 0.00048285063239430847.
237/237 [==============================] - 0s 

237/237 [==============================] - 0s 25us/step - loss: 38.0746 - accuracy: 0.0380
Epoch 893/1000

Epoch 00893: LearningRateScheduler setting learning rate to 0.00045109263812640143.
237/237 [==============================] - 0s 21us/step - loss: 38.0446 - accuracy: 0.0380
Epoch 894/1000

Epoch 00894: LearningRateScheduler setting learning rate to 0.0004502884222547128.
237/237 [==============================] - 0s 21us/step - loss: 38.0143 - accuracy: 0.0380
Epoch 895/1000

Epoch 00895: LearningRateScheduler setting learning rate to 0.0004494847325280887.
237/237 [==============================] - 0s 21us/step - loss: 37.9848 - accuracy: 0.0380
Epoch 896/1000

Epoch 00896: LearningRateScheduler setting learning rate to 0.0004486815979952052.
237/237 [==============================] - 0s 21us/step - loss: 37.9543 - accuracy: 0.0380
Epoch 897/1000

Epoch 00897: LearningRateScheduler setting learning rate to 0.00044787898960096766.
237/237 [==============================] - 0s 21

237/237 [==============================] - 0s 21us/step - loss: 36.8484 - accuracy: 0.0380
Epoch 936/1000

Epoch 00936: LearningRateScheduler setting learning rate to 0.0004170225923879726.
237/237 [==============================] - 0s 21us/step - loss: 36.8215 - accuracy: 0.0380
Epoch 937/1000

Epoch 00937: LearningRateScheduler setting learning rate to 0.00041624337989149305.
237/237 [==============================] - 0s 21us/step - loss: 36.7948 - accuracy: 0.0380
Epoch 938/1000

Epoch 00938: LearningRateScheduler setting learning rate to 0.0004154648095926552.
237/237 [==============================] - 0s 21us/step - loss: 36.7677 - accuracy: 0.0380
Epoch 939/1000

Epoch 00939: LearningRateScheduler setting learning rate to 0.0004146868524382791.
237/237 [==============================] - 0s 21us/step - loss: 36.7407 - accuracy: 0.0380
Epoch 940/1000

Epoch 00940: LearningRateScheduler setting learning rate to 0.00041390953747396866.
237/237 [==============================] - 0s 17

237/237 [==============================] - 0s 17us/step - loss: 35.7439 - accuracy: 0.0380
Epoch 979/1000

Epoch 00979: LearningRateScheduler setting learning rate to 0.0003841054037659666.
237/237 [==============================] - 0s 21us/step - loss: 35.7198 - accuracy: 0.0380
Epoch 980/1000

Epoch 00980: LearningRateScheduler setting learning rate to 0.0003833547898739192.
237/237 [==============================] - 0s 21us/step - loss: 35.6952 - accuracy: 0.0380
Epoch 981/1000

Epoch 00981: LearningRateScheduler setting learning rate to 0.0003826048761040188.
237/237 [==============================] - 0s 21us/step - loss: 35.6701 - accuracy: 0.0380
Epoch 982/1000

Epoch 00982: LearningRateScheduler setting learning rate to 0.000381855662452073.
237/237 [==============================] - 0s 17us/step - loss: 35.6459 - accuracy: 0.0380
Epoch 983/1000

Epoch 00983: LearningRateScheduler setting learning rate to 0.00038110717796067196.
237/237 [==============================] - 0s 17us

In [12]:
model.predict(X_test)

array([[ 3.5360096 ],
       [ 4.3990493 ],
       [ 8.96599   ],
       [11.273241  ],
       [10.421489  ],
       [15.8777275 ],
       [12.021342  ],
       [23.005232  ],
       [14.445108  ],
       [11.382313  ],
       [17.792772  ],
       [ 7.053015  ],
       [21.859144  ],
       [16.477543  ],
       [10.916637  ],
       [ 9.732147  ],
       [ 6.357668  ],
       [ 5.372998  ],
       [14.151054  ],
       [17.436474  ],
       [15.494333  ],
       [13.530794  ],
       [ 5.053394  ],
       [ 0.55392545],
       [11.110115  ],
       [12.289912  ],
       [ 8.438641  ],
       [12.8336525 ],
       [16.186972  ],
       [16.18926   ],
       [ 6.4882016 ],
       [ 9.973651  ],
       [14.2208605 ],
       [11.097471  ],
       [12.956579  ],
       [10.216654  ],
       [11.442198  ],
       [10.037359  ],
       [10.078008  ],
       [10.972787  ],
       [ 6.180884  ],
       [11.64784   ],
       [ 4.048981  ],
       [ 2.2961206 ],
       [10.544301  ],
       [14

In [13]:
print(model.predict(X_test)-y_test)

            G3
329 -10.463990
318  -5.600951
317  -0.034010
65   -3.726759
59   -5.578511
..         ...
276  13.758343
140   7.626267
142   3.912971
306  -6.462026
146   5.421498

[79 rows x 1 columns]
